# Driver History Tables

In [2]:
import pandas as pd
import copy
import warnings
import datetime as dt

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
# Import all the data
drivers_df = pd.read_csv("./f1db_csv/drivers.csv").drop(columns = "url")
results_df = pd.read_csv("./f1db_csv/results.csv")
constructors_df = pd.read_csv("./f1db_csv/constructors.csv")
races_df = pd.read_csv("./f1db_csv/races.csv")
qualifying_df = pd.read_csv("./f1db_csv/qualifying.csv")
d_standings_df = pd.read_csv("./f1db_csv/driver_standings.csv")

# Clean some names and create new variables
# drivers_df
drivers_df["number"] = drivers_df["number"].replace({r"\N": None})
drivers_df["driverName"] = drivers_df["forename"].str.cat(drivers_df["surname"],sep = " ")
drivers_df = drivers_df.drop(columns = ["forename", "surname"])

# results_df
results_df["position"] = results_df["position"].replace({r"\N": None})

# Merging to a full dataset for driver history
df_1 = pd.merge(drivers_df[["driverId", "driverName", "number", "nationality"]], results_df[["driverId", "raceId", "constructorId", "position", "fastestLapTime"]], on = "driverId")
df_2 = pd.merge(df_1, constructors_df[["constructorId", "name"]], on = "constructorId")
df_2 = df_2.rename({"position" : "racePosition"}, axis = "columns")
df_2 = df_2.rename({"name" : "constructorName"}, axis = "columns")
df_3 = pd.merge(df_2, d_standings_df[["driverId", "raceId", "points", "position", "wins"]], on = ["driverId", "raceId"])
df_3 = df_3.rename({"position" : "driverStanding"}, axis = "columns")
df_4 = pd.merge(df_3, races_df[["raceId", "year", "name", "date"]], on = "raceId")
df_5 = pd.merge(df_4, qualifying_df[["raceId", "driverId", "position", "q1", "q2", "q3"]], on = ["driverId", "raceId"])

for i in range(len(df_5['q1'])):
    nan_series = df_5.q1.isna()[i]
    if (df_5['q1'][i] == r"\N") | (nan_series == True):
        df_5['q1'][i] = None
        i += 1
    elif df_5['q1'][i] != 0:
        df_5['q1'][i] = float(str(df_5['q1'][i]).split(':')[1]) + (60 * float(str(df_5['q1'][i]).split(':')[0]))
        i += 1
    else:
        df_5['q1'][i] = None
        i += 1

for i in range(len(df_5['q2'])):
    nan_series = df_5.q2.isna()[i]
    if (df_5['q2'][i] == r"\N") | (nan_series == True):
        df_5['q2'][i] = None
        i += 1
    elif df_5['q2'][i] != 0:
        df_5['q2'][i] = float(str(df_5['q2'][i]).split(':')[1]) + (60 * float(str(df_5['q2'][i]).split(':')[0]))
        i += 1
    else:
        df_5['q2'][i] = None
        i += 1

for i in range(len(df_5['q3'])):
    nan_series = df_5.q3.isna()[i]
    if (df_5['q3'][i] == r"\N") | (nan_series == True):
        df_5['q3'][i] = None
        i += 1
    elif df_5['q3'][i] != 0:
        df_5['q3'][i] = float(str(df_5['q3'][i]).split(':')[1]) + (60 * float(str(df_5['q3'][i]).split(':')[0]))
        i += 1
    else:
        df_5['q3'][i] = None
        i += 1
        
for i in range(len(df_5['fastestLapTime'])):
    nan_series = df_5.fastestLapTime.isna()[i]
    if (df_5['fastestLapTime'][i] == r"\N") | (nan_series == True):
        df_5['fastestLapTime'][i] = None
        i += 1
    elif df_5['fastestLapTime'][i] != 0:
        df_5['fastestLapTime'][i] = float(str(df_5['fastestLapTime'][i]).split(':')[1]) + (60 * float(str(df_5['fastestLapTime'][i]).split(':')[0]))
        i += 1
    else:
        df_5['fastestLapTime'][i] = None
        i += 1

df_5["minQualifyingTime"] = df_5[["q1", "q2", "q3"]].min(skipna = True, axis = 1)
df_5 = df_5.drop(columns = ["q1", "q2", "q3"])
df_5.head(10)

,driverId,driverName,number,nationality,raceId,constructorId,racePosition,fastestLapTime,constructorName,points,driverStanding,wins,year,name,date,position,minQualifyingTime
0,1,Lewis Hamilton,44,British,18,1,1,87.452,McLaren,10.0,1,1,2008,Australian Grand Prix,2008-03-16,1,85.187
1,5,Heikki Kovalainen,None,Finnish,18,1,5,87.418,McLaren,4.0,5,0,2008,Australian Grand Prix,2008-03-16,3,85.452
2,2,Nick Heidfeld,None,German,18,2,2,87.739,BMW Sauber,8.0,2,0,2008,Australian Grand Prix,2008-03-16,5,85.518
3,3,Nico Rosberg,6,German,18,3,3,88.09,Williams,6.0,3,0,2008,Australian Grand Prix,2008-03-16,7,86.059
4,6,Kazuki Nakajima,None,Japanese,18,3,6,89.639,Williams,3.0,6,0,2008,Australian Grand Prix,2008-03-16,14,86.413
5,4,Fernando Alonso,14,Spanish,18,4,4,88.603,Renault,5.0,4,0,2008,Australian Grand Prix,2008-03-16,12,86.188
6,8,Kimi Räikkönen,7,Finnish,18,6,8,87.903,Ferrari,1.0,8,0,2008,Australian Grand Prix,2008-03-16,16,86.140
7,7,Sébastien Bourdais,None,French,18,5,7,89.534,Toro Rosso,2.0,7,0,2008,Australian Grand Prix,2008-03-16,18,87.446
8,1,Lewis Hamilton,44,British,19,1,5,95.462,McLaren,14.0,1,1,2008,Malaysian Grand Prix,2008-03-23,4,94.627
9,5,Heikki Kovalainen,None,Finnish,19,1,3,95.922,McLaren,10.0,4,0,2008,Malaysian Grand Prix,2008-03-23,3,94.759


In [74]:
# Clean this dataset: drop variables and rearrange
df = copy.deepcopy(df_5)
df = df.drop(columns = ["driverId", "constructorId"])
df = df[["driverName", "number", "nationality", "year", "name", "date", "constructorName", "position", "minQualifyingTime", "racePosition", "fastestLapTime", "wins", "points", "driverStanding"]]

# Turn date into datetime
df["date"] = pd.to_datetime(df["date"])

# Save it into a csv
df.to_csv("./f1db_csv/driver_history.csv")

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8181 entries, 0 to 8180
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   driverName         8181 non-null   object        
 1   number             4295 non-null   object        
 2   nationality        8181 non-null   object        
 3   year               8181 non-null   int64         
 4   name               8181 non-null   object        
 5   date               8181 non-null   datetime64[ns]
 6   constructorName    8181 non-null   object        
 7   position           8181 non-null   int64         
 8   minQualifyingTime  8052 non-null   float64       
 9   racePosition       6249 non-null   object        
 10  fastestLapTime     6220 non-null   object        
 11  wins               8181 non-null   int64         
 12  points             8181 non-null   float64       
 13  driverStanding     8181 non-null   int64         
dtypes: datet

In [77]:
# Function that chooses race history for a particular driver
def driver_history_selection(driver_name):
    driver_table = df[df.driverName == driver_name]
    driver_table = driver_table.sort_values("date", ascending = False)
    return driver_table

driver_history_selection("Alexander Albon")

,driverName,number,nationality,year,name,date,constructorName,position,minQualifyingTime,racePosition,fastestLapTime,wins,points,driverStanding
8142,Alexander Albon,23,Thai,2020,British Grand Prix,2020-08-02,Red Bull,12,86.545,8,88.689,0,26.0,6
8122,Alexander Albon,23,Thai,2020,Hungarian Grand Prix,2020-07-19,Red Bull,13,75.715,5,79.44,0,22.0,5
8102,Alexander Albon,23,Thai,2020,Styrian Grand Prix,2020-07-12,Red Bull,7,79.014,4,67.299,0,12.0,8
8082,Alexander Albon,23,Thai,2020,Austrian Grand Prix,2020-07-05,Red Bull,5,63.746,13,68.432,0,0.0,13
8064,Alexander Albon,23,Thai,2019,Abu Dhabi Grand Prix,2019-12-01,Red Bull,6,95.682,6,102.219,0,92.0,8
8044,Alexander Albon,23,Thai,2019,Brazilian Grand Prix,2019-11-17,Red Bull,6,67.935,14,71.087,0,84.0,8
8024,Alexander Albon,23,Thai,2019,United States Grand Prix,2019-11-03,Red Bull,6,92.548,5,98.029,0,84.0,6
8004,Alexander Albon,23,Thai,2019,Mexican Grand Prix,2019-10-27,Red Bull,5,75.336,5,79.325,0,74.0,8
7984,Alexander Albon,23,Thai,2019,Japanese Grand Prix,2019-10-13,Red Bull,6,87.851,4,92.775,0,64.0,8
7964,Alexander Albon,23,Thai,2019,Russian Grand Prix,2019-09-29,Red Bull,19,99.197,5,96.762,0,52.0,8
